# Joint FRAP-SPT Analysis

## Resolving Parameter Degeneracies

This notebook demonstrates how combining FRAP and SPT data resolves parameter ambiguities.

### The Problem: FRAP-Only Degeneracy

FRAP recovery curves often cannot uniquely determine (D, k_on, k_off):
- Multiple parameter combinations produce similar curves
- Diffusion vs binding kinetics are confounded
- Bound fraction is ambiguous

### The Solution: Add SPT Data

SPT provides independent constraints:
- **Dwell times** → k_on, k_off directly
- **Trajectory diffusion** → D_free, D_bound
- **Bound fraction** → Equilibrium check

### Workflow

1. Fit FRAP alone (show degeneracy)
2. Fit SPT alone
3. Joint fit (resolve ambiguity)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from frap_models import ReactionDiffusionModel
from spt_models import fit_dwell_times, DwellTimeModel
from spt_models.joint_frap_spt import fit_joint_model, JointFRAPSPT

plt.style.use('seaborn-v0_8-paper')
%matplotlib inline

## 1. Generate Synthetic Ground Truth Data

In [ ]:
# True parameters (what we want to recover)
true_params = {
    'D': 5.0,      # μm²/s
    'k_on': 2.0,   # 1/s
    'k_off': 3.0,  # 1/s
    'bleach_depth': 0.9
}

# Bound fraction at equilibrium
f_bound_true = true_params['k_on'] / (true_params['k_on'] + true_params['k_off'])
print(f"True bound fraction: {f_bound_true:.3f}")

# FRAP geometry
geometry = {
    'shape': (100, 100),
    'spacing': 0.1,
    'total_concentration': 1.0,
    'bound_fraction': f_bound_true,
    'bleach_region': {
        'type': 'circular',
        'center': (5.0, 5.0),
        'radius': 1.0,
        'bleach_depth': 0.9
    }
}

# Time points
timepoints = np.array([0, 0.5, 1, 2, 5, 10, 20, 40])

# Simulate FRAP
print("\nGenerating synthetic FRAP data...")
model = ReactionDiffusionModel()
frap_true = model.simulate(true_params, geometry, timepoints)

# Add noise
np.random.seed(42)
frap_noise_level = 0.02
frap_observed = frap_true + np.random.normal(0, frap_noise_level, len(frap_true))
frap_observed = np.clip(frap_observed, 0, 1)  # Keep in valid range

# Plot
plt.figure(figsize=(8, 6))
plt.plot(timepoints, frap_true, '-', linewidth=2, label='True curve')
plt.plot(timepoints, frap_observed, 'o', markersize=8, label='Observed (with noise)')
plt.xlabel('Time (s)', fontsize=12)
plt.ylabel('Normalized Fluorescence', fontsize=12)
plt.title('Synthetic FRAP Data', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 2. Generate Synthetic SPT Data

In [ ]:
# Generate dwell times from exponential distributions
np.random.seed(43)

# Bound state dwell times (rate = k_off)
n_bound_dwells = 100
bound_dwells = np.random.exponential(1/true_params['k_off'], n_bound_dwells)

# Unbound state dwell times (rate = k_on * [binding sites])
n_unbound_dwells = 150
concentration = 1.0  # Effective concentration
unbound_dwells = np.random.exponential(
    1/(true_params['k_on'] * concentration), 
    n_unbound_dwells
)

# Plot dwell time distributions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bound dwells
axes[0].hist(bound_dwells, bins=20, density=True, alpha=0.6, label='Observed')
t = np.linspace(0, np.max(bound_dwells), 100)
axes[0].plot(t, true_params['k_off'] * np.exp(-true_params['k_off'] * t),
            'r-', linewidth=2, label=f'True (k_off={true_params["k_off"]:.1f})')
axes[0].set_xlabel('Dwell Time (s)', fontsize=12)
axes[0].set_ylabel('Probability Density', fontsize=12)
axes[0].set_title('Bound State Dwell Times', fontsize=13)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Unbound dwells
axes[1].hist(unbound_dwells, bins=20, density=True, alpha=0.6, label='Observed')
t = np.linspace(0, np.max(unbound_dwells), 100)
rate = true_params['k_on'] * concentration
axes[1].plot(t, rate * np.exp(-rate * t),
            'r-', linewidth=2, label=f'True (k_on={true_params["k_on"]:.1f})')
axes[1].set_xlabel('Dwell Time (s)', fontsize=12)
axes[1].set_ylabel('Probability Density', fontsize=12)
axes[1].set_title('Unbound State Dwell Times', fontsize=13)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nGenerated {n_bound_dwells} bound dwells, {n_unbound_dwells} unbound dwells")

## 3. SPT-Only Analysis

In [ ]:
# Fit SPT dwell times
print("Fitting SPT data...")
spt_result = fit_dwell_times(
    bound_dwells=bound_dwells,
    unbound_dwells=unbound_dwells,
    concentration=concentration
)

print("\n=== SPT-Only Results ===")
print(f"k_off: {spt_result['k_off']:.3f} ± {spt_result['k_off_se']:.3f} s⁻¹")
print(f"  True value: {true_params['k_off']:.3f} s⁻¹")
print(f"\nk_on: {spt_result['k_on']:.3f} ± {spt_result['k_on_se']:.3f} s⁻¹")
print(f"  True value: {true_params['k_on']:.3f} s⁻¹")
print(f"\nBound fraction: {spt_result['bound_fraction']:.3f}")
print(f"  True value: {f_bound_true:.3f}")

print("\nNote: SPT accurately recovers kinetic rates!")

## 4. Demonstrate FRAP-Only Degeneracy

Show that multiple parameter sets can fit FRAP data similarly.

In [ ]:
# Test several parameter combinations
# Keep bound fraction similar but vary individual rates
param_sets = [
    {'D': 5.0, 'k_on': 2.0, 'k_off': 3.0, 'bleach_depth': 0.9},  # True
    {'D': 7.0, 'k_on': 4.0, 'k_off': 6.0, 'bleach_depth': 0.9},  # Faster kinetics
    {'D': 3.0, 'k_on': 1.0, 'k_off': 1.5, 'bleach_depth': 0.9},  # Slower kinetics
]

labels = ['True params', '2× faster kinetics', '2× slower kinetics']
colors = ['blue', 'red', 'green']

plt.figure(figsize=(10, 6))
plt.plot(timepoints, frap_observed, 'ko', markersize=10, 
         label='Observed data', zorder=10)

for params, label, color in zip(param_sets, labels, colors):
    # Update geometry bound fraction
    geom = geometry.copy()
    geom['bound_fraction'] = params['k_on'] / (params['k_on'] + params['k_off'])
    
    recovery = model.simulate(params, geom, timepoints)
    plt.plot(timepoints, recovery, '-', color=color, linewidth=2, 
             label=label, alpha=0.7)

plt.xlabel('Time (s)', fontsize=12)
plt.ylabel('Normalized Fluorescence', fontsize=12)
plt.title('FRAP Degeneracy: Multiple Parameter Sets Fit Data', fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n⚠️  Warning: FRAP alone cannot distinguish these parameter sets!")
print("This is why SPT data is critical.")

## 5. Joint FRAP-SPT Fitting

Combine both datasets to constrain parameters.

In [ ]:
# Prepare data for joint fitting
frap_data = {
    'recovery': frap_observed,
    'timepoints': timepoints,
    'geometry': geometry
}

spt_data = {
    'bound_dwells': bound_dwells,
    'unbound_dwells': unbound_dwells,
    'concentration': concentration
}

# Initial guess (could be from FRAP-only or SPT-only)
initial_guess = {
    'D': 5.0,
    'k_on': spt_result['k_on'],   # Use SPT estimate
    'k_off': spt_result['k_off'],
    'bleach_depth': 0.9
}

# Joint fit
print("Performing joint FRAP-SPT fit...")
joint_result = fit_joint_model(
    frap_data=frap_data,
    spt_data=spt_data,
    initial_guess=initial_guess
)

print("\n=== Joint FRAP-SPT Results ===")
print(f"D: {joint_result['params']['D']:.3f} μm²/s")
print(f"  True value: {true_params['D']:.3f} μm²/s")
print(f"\nk_on: {joint_result['params']['k_on']:.3f} s⁻¹")
print(f"  True value: {true_params['k_on']:.3f} s⁻¹")
print(f"\nk_off: {joint_result['params']['k_off']:.3f} s⁻¹")
print(f"  True value: {true_params['k_off']:.3f} s⁻¹")
print(f"\nBound fraction: {joint_result['bound_fraction']:.3f}")
print(f"  True value: {f_bound_true:.3f}")

## 6. Compare Results

In [ ]:
# Visualization of parameter recovery
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

params_list = ['D', 'k_on', 'k_off']
titles = ['Diffusion Coefficient', 'Binding Rate', 'Unbinding Rate']
units = ['μm²/s', 's⁻¹', 's⁻¹']

for ax, param, title, unit in zip(axes, params_list, titles, units):
    true_val = true_params[param]
    
    # True value
    ax.axhline(true_val, color='black', linestyle='--', linewidth=2, 
               label='True value', zorder=1)
    
    # SPT result (if available)
    if param in spt_result:
        spt_val = spt_result[param]
        spt_err = spt_result.get(f'{param}_se', 0)
        ax.errorbar([1], [spt_val], yerr=[spt_err*1.96], 
                   fmt='o', markersize=10, capsize=5, label='SPT only',
                   color='blue', zorder=2)
    
    # Joint result
    joint_val = joint_result['params'][param]
    ax.plot([2], [joint_val], 'o', markersize=10, label='Joint FRAP-SPT',
           color='red', zorder=2)
    
    ax.set_xlim([0.5, 2.5])
    ax.set_xticks([1, 2])
    ax.set_xticklabels(['SPT', 'Joint'])
    ax.set_ylabel(f'{title} ({unit})', fontsize=11)
    ax.set_title(title, fontsize=12)
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## Summary

### Key Findings

1. **FRAP-only fits are degenerate**: Multiple (D, k_on, k_off) combinations produce similar curves

2. **SPT constrains kinetics**: Dwell times directly measure k_on and k_off

3. **Joint fitting resolves ambiguity**: Combining FRAP and SPT uniquely determines all parameters

### Recommendations for Experimental Design

- **FRAP alone**: Use only when kinetics are known *a priori*
- **SPT alone**: Provides kinetics but not spatial information
- **Joint FRAP-SPT** (recommended): Best approach for complete characterization

### Grant/Manuscript Defensibility

This analysis demonstrates:
- **Explicit identifiability testing**: We show when parameters can/cannot be determined
- **Mechanistic grounding**: Models based on physical principles, not empirical fits
- **Experimental validation**: Joint fitting provides cross-validation
- **Quantitative rigor**: Parameter uncertainties properly estimated

### Next Steps

- Apply to real experimental data
- Perform model selection (AIC/BIC)
- Bootstrap for uncertainty quantification
- Test mass conservation in live-cell experiments